Controlflow Documentation: https://controlflow.ai/welcome

In [1]:
import controlflow as cf

from agent_utils import *
from agent_tools import *

/home/dplaia/Projekte/controlflow/.venv/lib/python3.13/site-packages/pydantic/fields.py:1042: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'env'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warn(
/home/dplaia/Projekte/controlflow/.venv/lib/python3.13/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [10]:
reasoningAgentChat = ChatHandler()
basicSearchAgent = BasicSearchAgent(perplexity_search=False)

def ask_reasoning_model(content: str) -> str:
    global reasoningAgentChat

    response = reasoningAgentChat(content)
    return response

def google_single_search(search_query: str) -> str:
    global basicSearchAgent
    result_text = basicSearchAgent(search_query)
    return result_text

def google_search(search_queries: list[str]) -> list[str]:
    results = []
    for query in search_queries:
        print(f"Search query: {query}")
        response = google_single_search(query)
        results.append(response)
    return results

def search_query_help(search_query: str) -> str:
    query = f"""
    We have to improve the search results given a user search query. 
    Please think of multiple google search queries (plain text) that would increase the quality of the search results, when we combine all the search results.

    # Desired output format:

    Google Search Queries:
    - Search query 1
    - Search query 2
    - etc.

    # User Search Query:
    {search_query}

    """
    
    text_response = reasoningAgentChat(query)
    return text_response

class SearchQueryAgentResponse(BaseModel):
    google_search_queries: list[str] = Field(description="The extracted google search queries (if available).")
    # google_scholar_queries: list[str] | None = Field(description="The extracted google scholar queries (if available).")
    # text_summary: str  | None = Field(description="Extract the text summary here (if available).")

searchQueryAgent = cf.Agent(
    name="Query",
    model = "google/gemini-2.0-flash",
    instructions=f"""
    Your goal is to extract search queries (e.g., google search, google scholar, etc.) that are mention in a text.
    """
)

def generate_research_report(user_search_query, result_text) -> str:
    global reasoningAgentChat

    response = reasoningAgentChat(f"""
        Please generate a high quality report text based on the search results (output in Markdown format).
        Add a reference section with citations at the end. Use the urls that are present in the input text, don't use other urls.

        Format:
        # References
        [1] http...
        [2] etc.

        Add the correct citations in the writen report. 

        The text should be as relevant to the user search query as possible. 

        # User Search Query:
        {user_search_query} 

        #Search Results (text):
        {result_text}
        
        """)

    return response

def get_search_result_text(results: list[str]) -> str:
    result_text = ""
    for k, result in enumerate(results):
        #console_print(result)
        result_text += f"""
        # Result query {k+1}:

        {result}

        """
    return result_text

def run_research(search_query: str, max_searches: int = 10) -> str:
    search_queries_text = search_query_help(search_query)
    console_print(search_queries_text)

    task_extract_search_queries = cf.Task(
        objective=f"Extract all search queries from the input text",
        instructions=f"Extract the google search and google scholar search queries from the text. Here is the input text:  {search_queries_text}",
        result_type=SearchQueryAgentResponse,
        agents=[searchQueryAgent]
    )

    result = task_extract_search_queries.run()
    results = google_search(result.google_search_queries[:max_searches])
    result_text = get_search_result_text(results)
    response = generate_research_report(search_query, result_text)
    
    return response

In [ ]:
search_query = "What can you tell me about the new technology or system from BYD called 'God's Eye'?"

response = run_research(search_query)

console_print(response)